In [8]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set dataset directory
os.environ['TFDS_DATA_DIR'] = 'C:/Users/<your_user>/tensorflow_datasets'
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 128

# MNIST Dataset
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label


ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache().shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(AUTOTUNE)

model = keras.Sequential(
    [
        keras.Input((28, 28, 1)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Flatten(),
        layers.Dense(10, activation="softmax"),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

model.fit(ds_train, epochs=5, verbose=2)
model.evaluate(ds_test)

# IMDb Dataset
(ds_train_imdb, ds_test_imdb), ds_info_imdb = tfds.load(
    "imdb_reviews",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

tokenizer = tfds.features.text.Tokenizer()


def build_vocabulary():
    vocabulary = set()
    for text, _ in ds_train_imdb:
        vocabulary.update(tokenizer.tokenize(text.numpy().decode('utf-8').lower()))
    return vocabulary


vocabulary = build_vocabulary()

encoder = tfds.features.text.TokenTextEncoder(
    list(vocabulary), oov_token="<UNK>", lowercase=True, tokenizer=tokenizer
)


def my_enc(text_tensor, label):
    encoded_text = encoder.encode(text_tensor.numpy().decode('utf-8'))
    return encoded_text, label


def encode_map_fn(text, label):
    encoded_text, label = tf.py_function(
        my_enc, inp=[text, label], Tout=(tf.int64, tf.int64)
    )
    encoded_text.set_shape([None])
    label.set_shape([])
    return encoded_text, label


ds_train_imdb = ds_train_imdb.map(encode_map_fn, num_parallel_calls=AUTOTUNE)
ds_train_imdb = ds_train_imdb.cache().shuffle(1000).padded_batch(32).prefetch(AUTOTUNE)

ds_test_imdb = ds_test_imdb.map(encode_map_fn, num_parallel_calls=AUTOTUNE)
ds_test_imdb = ds_test_imdb.padded_batch(32).prefetch(AUTOTUNE)

model = keras.Sequential(
    [
        layers.Masking(mask_value=0),
        layers.Embedding(input_dim=len(vocabulary) + 2, output_dim=32),
        layers.GlobalAveragePooling1D(),
        layers.Dense(64, activation="relu"),
        layers.Dense(1),
    ]
)

model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(3e-4, clipnorm=1),
    metrics=["accuracy"],
)

model.fit(ds_train_imdb, epochs=15, verbose=2)
model.evaluate(ds_test_imdb)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling...:   0%|          | 0/10 [00:00<?, ? shard/s]

...

0 examples [00:00, ? examples/s]

Shuffling...:   0%|          | 0/1 [00:00<?, ? shard/s]

Reading...: 0 examples [00:00, ? examples/s]

Writing...:   0%|          | 0/10000 [00:00<?, ? examples/s]

UnknownError: Failed to rename: C:\Users\Ramsaheb Prasad\tensorflow_datasets\mnist\1.0.0.incompleteN8QUUZ to: C:\Users\Ramsaheb Prasad\tensorflow_datasets\mnist\1.0.0 : Access is denied.
; Input/output error